**2483. Minimum Penalty for a Shop**

**Medium**

**Companies**: META

You are given the customer visit log of a shop represented by a 0-indexed string customers consisting only of characters 'N' and 'Y':

- if the ith character is 'Y', it means that customers come at the ith hour

- whereas 'N' indicates that no customers come at the ith hour.

If the shop closes at the jth hour (0 <= j <= n), the penalty is calculated as follows:

- For every hour when the shop is open and no customers come, the penalty increases by 1.

- For every hour when the shop is closed and customers come, the penalty increases by 1.

Return the earliest hour at which the shop must be closed to incur a minimum penalty.

**Note** that if a shop closes at the jth hour, it means the shop is closed at the hour j.

**Example 1:**

```python
Input: customers = "YYNY"
Output: 2
```

**Explanation:**

- Closing the shop at the 0th hour incurs in 1+1+0+1 = 3 penalty.
- Closing the shop at the 1st hour incurs in 0+1+0+1 = 2 penalty.
- Closing the shop at the 2nd hour incurs in 0+0+0+1 = 1 penalty.
- Closing the shop at the 3rd hour incurs in 0+0+1+1 = 2 penalty.
- Closing the shop at the 4th hour incurs in 0+0+1+0 = 1 penalty.
- Closing the shop at 2nd or 4th hour gives a minimum penalty. Since 2 is earlier, the optimal closing time is 2.

**Example 2:**

```python
Input: customers = "NNNNN"
Output: 0
```

**Explanation:** It is best to close the shop at the 0th hour as no customers arrive.

**Example 3:**

```python
Input: customers = "YYYY"
Output: 4
```

**Explanation:** It is best to close the shop at the 4th hour as customers arrive at each hour.

**Constraints:**

- 1 <= customers.length <= 105
- customers consists only of characters 'Y' and 'N'.


In [ ]:
class Solution:
    def bestClosingTime(self, customers: str) -> int:
        """
        ALGORITHM (BRUTE FORCE):

        1. Let n = length of customers.
        2. Try every possible closing hour j from 0 to n.
        3. For each j:
           - Shop is OPEN during hours [0, j)
             → Every 'N' adds 1 penalty.
           - Shop is CLOSED during hours [j, n)
             → Every 'Y' adds 1 penalty.
        4. Total penalty = open penalties + closed penalties.
        5. Track the minimum penalty and the earliest hour.
        6. Return the best closing hour.

        Time Complexity: O(n^2)
        Space Complexity: O(1)
        """

        n = len(customers)
        min_penalty = float('inf')
        best_hour = 0

        for j in range(n + 1):
            penalty = 0

            # Open hours [0, j)
            for i in range(j):
                if customers[i] == 'N':
                    penalty += 1

            # Closed hours [j, n)
            for i in range(j, n):
                if customers[i] == 'Y':
                    penalty += 1

            if penalty < min_penalty:
                min_penalty = penalty
                best_hour = j

        return best_hour


In [ ]:
class Solution:
    def bestClosingTime(self, customers: str) -> int:
        """
        ALGORITHM (PREFIX + SUFFIX):

        1. prefixN[i]:
           - Number of 'N' in customers[0 : i]
           - Penalty when shop is OPEN.

        2. suffixY[i]:
           - Number of 'Y' in customers[i : n]
           - Penalty when shop is CLOSED.

        3. For each closing hour j:
           penalty(j) = prefixN[j] + suffixY[j]

        4. Choose earliest j with minimum penalty.

        Time Complexity: O(n)
        Space Complexity: O(n)
        """

        n = len(customers)

        prefixN = [0] * (n + 1)
        suffixY = [0] * (n + 1)

        # Build prefix array
        for i in range(n):
            prefixN[i + 1] = prefixN[i]
            if customers[i] == 'N':
                prefixN[i + 1] += 1

        # Build suffix array
        for i in range(n - 1, -1, -1):
            suffixY[i] = suffixY[i + 1]
            if customers[i] == 'Y':
                suffixY[i] += 1

        min_penalty = float('inf')
        best_hour = 0

        for j in range(n + 1):
            penalty = prefixN[j] + suffixY[j]
            if penalty < min_penalty:
                min_penalty = penalty
                best_hour = j

        return best_hour


In [ ]:
class Solution:
    def bestClosingTime(self, customers: str) -> int:
        """
        ALGORITHM (GREEDY / RUNNING PENALTY):

        1. Assume shop closes at hour 0.
           - Shop is closed all the time.
           - Initial penalty = count of 'Y'.

        2. Move closing hour from left to right.
           For each hour passed:
           - 'Y' → penalty decreases by 1 (customer is now served)
           - 'N' → penalty increases by 1 (open with no customer)

        3. Track the minimum penalty.
        4. Return the earliest hour with minimum penalty.

        Time Complexity: O(n)
        Space Complexity: O(1)
        """

        penalty = customers.count('Y')
        min_penalty = penalty
        best_hour = 0

        for hour in range(1, len(customers) + 1):
            if customers[hour - 1] == 'Y':
                penalty -= 1
            else:
                penalty += 1

            if penalty < min_penalty:
                min_penalty = penalty
                best_hour = hour

        return best_hour


In [ ]:
class Solution:
    def bestClosingTime(self, customers: str) -> int:
        """
        ALGORITHM (GREEDY SCORE-BASED APPROACH):

        1. Think of each hour as a decision:
           - If we keep the shop OPEN during a 'Y' hour,
             we avoid a penalty → GOOD (+1 score).
           - If we keep the shop OPEN during an 'N' hour,
             we get a penalty → BAD (-1 score).

        2. Convert the problem into a score maximization problem:
           - 'Y' contributes +1
           - 'N' contributes -1

        3. Start with the shop CLOSED at hour 0:
           - current = 0  → running score
           - max_score = 0 → best score seen so far
           - best_hour = 0 → best closing hour

        4. Traverse the customers string from left to right:
           - Update the running score:
               current += +1 if 'Y'
               current += -1 if 'N'
           - If the running score becomes greater than max_score:
               • Update max_score
               • Set best_hour = i + 1
                 (closing after this hour is better)

        5. After processing all hours,
           return the earliest hour with the maximum score.

        6. This works because maximizing this score
           is equivalent to minimizing the total penalty.

        Time Complexity: O(n)
        Space Complexity: O(1)
        """

        best_hour = 0
        current = 0
        max_score = 0

        for i, c in enumerate(customers):
            # +1 benefit for 'Y', -1 cost for 'N'
            current += 1 if c == 'Y' else -1

            # Update best closing hour if score improves
            if current > max_score:
                max_score = current
                best_hour = i + 1

        return best_hour
